In [1]:
def invert_mod_q(f):
    # f in ZZ[x], f^(-1) in ZZ[x]
    # ((f * f^(-1)) % (x^n + 1)) % q = 1
    
    T = Zx.change_ring(Integers(q)).quotient(x^n+1)
    return Zx(lift(1/T(f)))

In [2]:
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler
from math import sqrt

def generate_trapdoor():
    # generate trapdoor matrix B, list[[]] representation
    magic_constant = [k * (0.1 + 0.002 * i) for i in range(k)]

    B = [[None for j in range(k + 1)] for i in range(k + 1)]

    eta = 1
    for i in range(k):
        for j in range(k + 1):
            sigma = magic_constant[i] * (q ** (1 / (k + 1)))
            sig = sqrt(1 / (n * (k + 1 - i))) * sigma
            B[i][j] = DiscreteGaussianDistributionPolynomialSampler(Zx, n, sig)()
    
    return B

In [3]:
def submatrix(M, row, col):
    nrow = len(M)
    ncol = len(M[0])
    rep = [[M[i][j] for j in range(ncol) if j != col] for i in range(nrow) if i != row]
    return rep

In [4]:
def matrix_invert_mod_q(f):
    # matrix: f in ZZ[x], f^(-1) in ZZ[x]
    # ((f * f^(-1)) % (x^n+1)) % q = 1
    
    f_det = f.determinant()
    f_det_inv = invert_mod_q(f_det)
    f_adj = f.adjugate()
    f_inv = f_adj * f_det_inv
    return f_det, f_adj, f_inv

In [5]:
def neg_mat(M):
    # M in [[]]
    # calculate the negative of a matrix
    return [[-M[i][j] for j in range(k)] for i in range(k)]

In [6]:
def xgcd_mod(f, g):
    # uf * f = df mod (x^n + 1)
    # ug * g = dg mod (x^n + 1)
    # df, dg in Z
    # uf, ug in Z[x]/(x^n + 1)
    df, uf, vf = (f % (x^n + 1)).xgcd(x^n + 1)
    dg, ug, vg = (g % (x^n + 1)).xgcd(x^n + 1)
    return df, dg, uf, ug

In [7]:
def to_bar(f):
    return Zx([f[0]]+[-f[n - i] for i in range(1, n)])

In [8]:
def cal_FG(f, g):
    rf, rg, _pf, _pg = xgcd_mod(f, g)
    d, u, v = rf.xgcd(rg)
    F = (q * v * (-1) * _pg) % (x^n + 1)
    G = (q * u * _pf) % (x^n + 1)
    return F, G

In [9]:
def convert_to_Zx(f):
    return Zx([int(i) for i in f])

In [10]:
def round_(f, num):
    # f in Zx
    # round(f/num) in Z[x]/(x^n + 1)
    return Zx([round(list(f)[i]/num) for i in range(n)])

In [11]:
def cal_ele_norm(ele):
    return math.sqrt(sum([float(i)**2 for i in ele]))

In [12]:
def cal_vec_norm(vec):
    return math.sqrt(sum([cal_ele_norm(ele)**2 for ele in vec]))

In [13]:
def cal_mat_norm(mat):
    return max([cal_vec_norm(vec) for vec in mat])

In [14]:
n = 2**2
q = random_prime(n**8, lbound=n**7)
k = 0
while 2 ** k < q:
    k += 1
    
Zx.<x> = ZZ[]

print(f"n={n}")
print(f"q={q}")
print(f"k={k}")

n=4
q=44579
k=16


In [15]:
max_gs_norm = k * float(q ** (1 / (k + 1)))

In [27]:
def mod_ntru_gen():
    while(true):
        B = generate_trapdoor()
        f = matrix(Zx, neg_mat(submatrix(B, k, 0)))
        g = matrix(Zx, k, 1, [B[j][0] for j in range(k)])
        try:
            f_det, f_adj, f_inv = matrix_invert_mod_q(f)  # f^(-1) mod q
        except (ZeroDivisionError, ValueError):
            continue
            
        h = ((f_inv * g) % (x^n + 1)) % q  # h = f^(-1) * g mod q

        # generate A = [1 h]
        A = [None for j in range(k + 1)]  
        for j in range(1, k + 1):
            A[j] = h[j - 1][0]
        A[0] = Zx([1])

        # generate F, G by f, g
        g_adjf = Zx(list((f_adj * g) % (x^n + 1))[0][0])
        rf, rg, _pf, _pg = xgcd_mod(f_det, g_adjf)
        
        if gcd(rf, rg)!=1 or gcd(rf, q)!=1:
            continue
            
        F, G = cal_FG(f_det, g_adjf)

        # reduce F, G by F = F-k * f, G = G - k * g
        ffgg = (f_det * to_bar(f_det) + g_adjf * to_bar(g_adjf)) % (x^n + 1)
        fFgG = (F * to_bar(f_det) + G * to_bar(g_adjf)) % (x^n + 1)
        dfg, ufg, vfg = ffgg.xgcd(x^n + 1)
        k_fg = round_(convert_to_Zx((ufg * fFgG) % (x^n + 1)), int(dfg))
        F_ = F - (k_fg * (f_det % (x^n + 1))) % (x^n + 1)
        G_ = G - (k_fg * (g_adjf % (x^n + 1))) % (x^n + 1)

        # insert F, G into trapdoor matrix B
        B[k][0] = G_
        B[k][1] = -F_
        for i in range(2, k + 1):
            B[k][i] = Zx([0])
            
        print(B)
        # transform A, B from list to ring matrix in Z[x]/(x^n + 1)
        A_matZ = matrix(Zx, k + 1, 1, A)
        B_matZ = matrix(Zx, k + 1, k + 1, B)

        return A_matZ, B_matZ

        


In [29]:
A, B = mod_ntru_gen()

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, x^2, 0, 0, -x^3, 0], [0, 0, 0, x^2, 0, 0, 0, 0, 0, 0, 0, x^3, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, -x^3, 0, -x, 0, -x, x^3 - 1, 0, x^3 - 1, 0, 0, 0], [x^2, 0, -x^3, 0, 0, -x^2, x, 0, 0, 0, 0, 0, 0, 0, -x, 1, x^2], [0, 0, x^3 + x^2, x^3, -1, 0, 0, 0, 0, 0, 0, -x^3, 0, 0, -x, -x^3 - x, 0], [1, x, x^3 + x^2, 0, 0, -x^3, 0, 0, 0, x^2, 0, x^2, -x^3 + x, x^3, 0, 0, x^2], [x^2, x - 1, 0, -x^3 + 1, 0, 0, -x^3 + x - 1, 0, -1, 0, x^2 + 1, x^3 + x^2, -x^3 + 1, x^2, -x^2 + 1, x^3, -x^3 + x], [0, x^2, 0, -x, -x^2, x^3 - 1, x, -1, 0, -x^2, 0, x^3 + 1, -x^3, 0, x^2 - x, x^2, 1], [0, x^3 - x, -x, 0, 1, -1, -x^3 + x, x, 0, -x^3, -x^3 - 1, x^2 + 1, x^2 + x + 1, 0, 1, x^2, 0], [-x^3 - x^2, x^3 + x, -x^2 + x + 1, -x^2 + x, x^2 - x, -x^2 + x, x^3 - 1, 1, -x^2, -1, -x^2 - x, -2*x^3 - 1, -x^2, x^3 - x^2 - 1, 0, -x^2 + x, -x^2 - 1], [-x^2 + x + 1, 1, x^3 - x, 0, x^3 - x, x, -x^2, x^3, x + 1, x^2, x^3 + x^2, 0, x^2 + 1, -x^3 - x, x^3 + x^2 - 1, x + 1, -x^3], [0, -x^3 + x^2 

In [30]:
A

[                                      1]
[40665*x^3 + 38283*x^2 + 18236*x + 42942]
[12122*x^3 + 37303*x^2 + 22573*x + 20652]
[ 34050*x^3 + 36483*x^2 + 18460*x + 9853]
[22799*x^3 + 26139*x^2 + 16580*x + 24926]
[ 39824*x^3 + 34714*x^2 + 1392*x + 29216]
[  42142*x^3 + 19135*x^2 + 3894*x + 6665]
[ 26253*x^3 + 21347*x^2 + 7546*x + 10442]
[  35853*x^3 + 3200*x^2 + 18362*x + 6301]
[ 43630*x^3 + 24151*x^2 + 4546*x + 21102]
[ 2424*x^3 + 37021*x^2 + 32119*x + 36349]
[  31889*x^3 + 3825*x^2 + 38866*x + 5466]
[20653*x^3 + 22036*x^2 + 37875*x + 30770]
[ 40210*x^3 + 4895*x^2 + 18057*x + 24361]
[ 44340*x^3 + 25748*x^2 + 17255*x + 7816]
[  6251*x^3 + 8193*x^2 + 13805*x + 35451]
[   623*x^3 + 5778*x^2 + 22406*x + 22261]

In [31]:
B

[                                              1                                               0                                               0                                               0                                               0                                               0                                               0                                               0                                               0                                               0                                              -1                                               0                                             x^2                                               0                                               0                                            -x^3                                               0]
[                                              0                                               0                                               0                                      

In [32]:
((B * A) % (x^n + 1)) % q

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]

In [33]:
(B.determinant()) % (x^n + 1)

44579

In [26]:
max_gs_norm

30.032966861938426

In [36]:
from sage.modules.misc import gram_schmidt
Qx.<x> = QQ[]
S = Qx.quotient(x^n + 1)
B_matQ = matrix(S, k + 1, k + 1, B)
try:
    G, mu = gram_schmidt(B_matQ.rows())
    print("GS norm =", cal_mat_norm(G))
except (ZeroDivisionError, ValueError):
    print(1)

GS norm = 29.318771982156832
